In [1]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk import * #bigrams, trigrams
import numpy as np
from collections import Counter, defaultdict
import string
import re
import json
import ast

In [2]:
# read in tweets from Erin's DBs
t1 = pd.read_csv("scraped_tweets1.csv")
t2 = pd.read_csv("scraped_tweets2.csv")
t3 = pd.read_csv("scraped_tweets3.csv")
t4 = pd.read_csv("scraped_tweets4.csv")

In [3]:
frames = [t1, t2, t3, t4]
tweets_erinDB = pd.concat(frames)

In [4]:
d = []
with open('usernames_filtered_2.txt') as rf:
    lines = rf.readlines()
    for line in lines:
        d.append(line[:-2])

In [5]:
tweets_erinDB_filtered = tweets_erinDB[tweets_erinDB['query'].isin(d)]

In [6]:
# read in tweets from Nathan's DB
t5 = pd.read_csv("scraped_tweets5.csv")

In [7]:
frames2 = [tweets_erinDB_filtered, t5]
tweets = pd.concat(frames2)

In [8]:
# remove non-english tweets, reset indices
tweets_eng = tweets[tweets['language'].isin(['en', 'und'])]
tweets_eng = tweets_eng.reset_index(drop = True)

In [9]:
# filter users who have less than 100 tweets
tweets_eng['count'] = 1
tweet_count_grouped = tweets_eng.groupby('query')
tweet_count = tweet_count_grouped['count'].agg([np.sum])
tweet_count['keep'] = tweet_count['sum'] >= 100
tweet_keep = tweet_count[tweet_count['keep'] == True]
users_tweet_keep = list(tweet_keep.index)
tweets_eng_keep = tweets_eng[tweets_eng['query'].isin(users_tweet_keep)]

#len(tweets_eng_keep), len(tweets_eng)

In [10]:
# filter users who have 3x as many friends as followers (potential bots)
friend_count_grouped = tweets_eng_keep.groupby('query')
followers_count = friend_count_grouped['num_followers'].agg([np.mean])
friends_count = friend_count_grouped['num_friends'].agg([np.mean])
friends_count['ratio'] = friends_count['mean']/friends_count['mean']
friends_count['keep'] = friends_count['ratio'] <= 3
ratio_keep = friends_count[friends_count['keep'] == True]
users_ratio_keep = list(ratio_keep.index)
tweets_trimmed = tweets_eng_keep[tweets_eng_keep['query'].isin(users_ratio_keep)]
#len(tweets_trimmed), len(tweets_eng_keep), len(tweets_eng)
tweets_trimmed = tweets_trimmed.reset_index(drop = True)

In [11]:
# deal with NaN
tweets_trimmed['entities_hashtags'] = tweets_trimmed['entities_hashtags'].fillna(0)
tweets_trimmed['entities_urls'] = tweets_trimmed['entities_urls'].fillna(0)

In [12]:
users = list(set(tweets_trimmed['query']))
user_descriptions = {}

In [13]:
with open('username_genders_erinCorpus.txt') as rf:
    descriptions = rf.readlines()

for line in descriptions:
    cool = line.split('\t')
    user_descriptions[cool[0]] = [cool[1][:-1]]

In [14]:
with open('users_refined1_keywords.json', 'r') as rf:
    descriptions = rf.readlines()

for i in range(len(descriptions)):
    line = descriptions[i].split(', ', 1)
    user = line[0]
    keyword_string = line[1][:-3]
    keyword_list = ast.literal_eval(keyword_string)
    user_descriptions[user] = [key.lower() for key in keyword_list]

In [15]:
with open('users_refined2_keywords.json', 'r') as rf:
    descriptions = rf.readlines()

for i in range(len(descriptions)):
    line = descriptions[i].split(', ', 1)
    user = line[0]
    keyword_string = line[1][:-3]
    keyword_list = ast.literal_eval(keyword_string)
    user_descriptions[user] = [key.lower() for key in keyword_list]

In [16]:
with open('users_refined3_keywords.json', 'r') as rf:
    descriptions = rf.readlines()

for i in range(len(descriptions)):
    line = descriptions[i].split(', ', 1)
    user = line[0]
    keyword_string = line[1][:-3]
    keyword_list = ast.literal_eval(keyword_string)
    user_descriptions[user] = [key.lower() for key in keyword_list]

In [17]:
with open('users_refined4_keywords.json', 'r') as rf:
    descriptions = rf.readlines()

for i in range(len(descriptions)):
    line = descriptions[i].split(', ', 1)
    user = line[0]
    keyword_string = line[1][:-3]
    keyword_list = ast.literal_eval(keyword_string)
    user_descriptions[user] = [key.lower() for key in keyword_list]

In [18]:
with open('users1_keywords.json') as rf:
    descriptions = rf.readlines()

for i in range(len(descriptions)):
    line = descriptions[i].split(', ', 1)
    user = line[0]
    keyword_string = line[1][:-3]
    keyword_list = ast.literal_eval(keyword_string)
    user_descriptions[user] = [key.lower() for key in keyword_list]

In [19]:
user_descriptions

{'moubarek73': ['gender'],
 'Aletzz_619': ['man'],
 'shantie_d': ['woman', 'queer'],
 'lorenzoology': ['trans'],
 'ForeverMoreVids': ['mom'],
 'kellydonlin': ['girl'],
 'whoopy07': ['man'],
 'SDMilitaryWife': ['mother'],
 'cheetahkin': ['gender'],
 'tmj_was_manuf': ['man'],
 'Paul_Convery': ['dad'],
 'kuchidake_onna': ['trans'],
 'rltaylor22': ['man'],
 'eleanor_heap': ['girl'],
 'GiniMomof9': ['mom'],
 'tmj_mad_manuf': ['man'],
 'belovedtaozi': ['queer'],
 'InsourceInc': ['trans'],
 'knzconnor': ['trans', 'queer'],
 'Frgluver28': ['mom'],
 'iMixShows901': ['man'],
 'KathrynMarquis': ['mother'],
 'brianjdixon': ['dad'],
 'MADFABstyle': ['girl'],
 'gregawashington': ['dad'],
 '_ignaciolm': ['man'],
 'vitatrain4life': ['mom'],
 'TyronWilson': ['queer'],
 '_scaryh': ['woman', 'trans'],
 'polopirate': ['man'],
 'NiCOLE_SANDRA': ['girl'],
 'Pastordude': ['father'],
 'SavvyLeigh87': ['mother'],
 'photolarry': ['queer'],
 'Maka_Aruban': ['man'],
 'JenevaMarieeee': ['man'],
 'LaundryDayBlues':

In [19]:
# scheme 1
m_keys = ['dad', 'father', 'brother', 'uncle', 'grampa', 'grandpa', 'grandfather', 'papa', 'male', 'boy', 'man']
f_keys = ['mom', 'mother', 'sister', 'aunt', 'gramma', 'grandma', 'grandmother', 'mama', 'female', 'girl', 'woman']
t_keys = ['trans']
tm_keys = ['ftm']
tf_keys = ['mtf']
o_keys = ['queer', 'fluid', 'binary']
#keywords = ['man', 'woman']
#for i in users:
    #keywords = user_dict[i]
labels = {}    

for user, keywords in user_descriptions.iteritems():
    if not set(f_keys).isdisjoint(keywords) and not set(m_keys).isdisjoint(keywords):
#        print 'M and F gender keywords present'
        identity = 'unknown'
        pass
    else:

        if 'trans' in keywords:
            if not set(tf_keys).isdisjoint(keywords):
                identity = 'transfemale'
            elif not set(tm_keys).isdisjoint(keywords):
                identity = 'transmale'
            else:
                if not set(f_keys).isdisjoint(keywords):
                    identity = 'transfemale'
                elif not set(m_keys).isdisjoint(keywords):
                    identity = 'transmale'    
                else:
                    if 'fluid' in keywords:
                        identity = 'genderfluid'
                    elif 'binary' in keywords:
                        identity = 'nonbinary'
                    elif 'queer' in keywords:
                        identity ='genderqueer'
                    else:
                        identity = 'transgender' #may just say genderqueer to catch all that fall here

        elif not set(o_keys).isdisjoint(keywords):
            if 'fluid' in keywords:
                identity = 'genderfluid'
            elif 'gender' in keywords or 'queer' in keywords:
                identity = 'genderqueer'
            elif 'binary' in keywords:
                identity = 'nonbinary'

        elif not set(f_keys).isdisjoint(keywords):
            identity = 'female'
        elif not set(m_keys).isdisjoint(keywords):
            identity = 'male'
        else:
            identity = 'unknown'
#            print 'No identity determined'
            pass

    if identity == 'genderfluid' or identity == 'nonbinary' or identity == 'genderqueer' or identity == 'transgender':
        identity = 'other'

    labels[user] = identity
#    print identity

In [20]:
labels

{'moubarek73': 'unknown',
 'Aletzz_619': 'male',
 'shantie_d': 'other',
 'ForeverMoreVids': 'female',
 'Vava_MCall': 'unknown',
 'whoopy07': 'male',
 'SDMilitaryWife': 'female',
 'karacantrell': 'female',
 'Paul_Convery': 'male',
 'kuchidake_onna': 'other',
 'rltaylor22': 'male',
 'eleanor_heap': 'female',
 'GiniMomof9': 'female',
 'tmj_mad_manuf': 'male',
 'belovedtaozi': 'other',
 'InsourceInc': 'other',
 'knzconnor': 'other',
 'Frgluver28': 'female',
 'iMixShows901': 'male',
 'KathrynMarquis': 'female',
 'brianjdixon': 'male',
 'MADFABstyle': 'female',
 'IamXiaoChen': 'male',
 '_ignaciolm': 'male',
 'vitatrain4life': 'female',
 '_scaryh': 'transfemale',
 'polopirate': 'male',
 'NiCOLE_SANDRA': 'female',
 'Pastordude': 'male',
 'SavvyLeigh87': 'female',
 'photolarry': 'other',
 'Maka_Aruban': 'male',
 'JenevaMarieeee': 'male',
 'StephRomanovich': 'female',
 'erell_briendo': 'unknown',
 'JStormspotter': 'unknown',
 'daddicusbinch': 'other',
 'tomkelly_ussa': 'male',
 'vertigoxoxo': 'o

In [28]:
unknowns = 0
for label in labels.values():
    if label == "unknown":
        unknowns += 1

print unknowns

1185


In [29]:
with open('user_gender_labels.txt', 'w') as wf:
    json.dump(labels, wf)